In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt

import random

import tensorflow as tf
from keras.optimizers import *
from keras.layers import *
from keras.models import *
from keras.regularizers import l2

from sklearn.utils import shuffle

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

NUM_CLASSES = 5


# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


In [2]:
def loadfiles():
    imgs = []
    labels = []
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            file = os.path.join(dirname, filename)
            ext = filename.split('.')[1]
            if ext == 'csv':
                label = int(filename.split('_')[1])
                imgs.append(np.loadtxt(open(file, "rb"), delimiter=",", skiprows=1))
                labels.append(label)
    return np.array(imgs), np.array(labels)
            

In [3]:
imgs, labels = loadfiles()
indices = np.array(list(range(imgs.shape[0])))
np.random.shuffle(indices)
test_imgs = imgs[indices[:1000]]
test_labels = labels[indices[:1000]]
train_imgs = imgs[indices[1000:]]
train_labels = labels[indices[1000:]]

In [4]:
train_arr = [[] for i in range(NUM_CLASSES)]
for idx, i in enumerate(train_labels):
    train_arr[i].append(idx)
test_arr = [[] for i in range(NUM_CLASSES)]
for idx, i in enumerate(test_labels):
    test_arr[i].append(idx)

In [5]:
def get_batch(batch_size, s="train"):
    global NUM_CLASSES, train_imgs, test_imgs, train_labels, test_labels, train_arr, test_arr
    if s == "train":
        imgs = train_imgs
        labels = train_labels
        arr = train_arr
    else:
        imgs = test_imgs
        labels = test_labels
        arr = test_arr
        
    
    n_classes = NUM_CLASSES
    n_examples, w, h = imgs.shape
    pairs = [np.zeros((batch_size, w, h, 1)) for i in range(2)]
    targets = np.zeros((batch_size,))
    
    targets[batch_size//2:] = 1
    
    categories = np.random.randint(0, n_classes, batch_size)
    
    for i in range(batch_size):
        category = categories[i]
        idx_1 = random.choice(arr[category])
        pairs[0][i,:,:,:] = imgs[idx_1].reshape(w, h, 1)
        if i >= batch_size // 2:
            category_2 = category  
        else: 
            category_2 = (category + np.random.randint(1,n_classes)) % n_classes
        idx_2 = random.choice(arr[category_2])
        
        pairs[1][i,:,:,:] = imgs[idx_2].reshape(w, h, 1)
    
    return pairs, targets
        
    
    

In [6]:
def generate(batch_size, s="train"):
    while True:
        pairs, targets = get_batch(batch_size, s)
        yield (pairs, targets)

In [7]:
def get_siamese_model(input_shape):
    """
        Model architecture
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer='random_normal', kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer='random_normal',
                     bias_initializer='random_normal', kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer='random_normal',
                     bias_initializer='random_normal', kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer='random_normal',
                     bias_initializer='random_normal', kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer='random_normal',bias_initializer='random_normal'))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer='random_normal')(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [8]:
model = get_siamese_model(tuple(list(train_imgs[0].shape)+[1]))
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if(filename == 'epoch150-weights.h5'):
            file = os.path.join(dirname, filename)
            model.load_weights(file)

optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)
    


In [9]:
def make_oneshot_task(N, s="val"):
    global NUM_CLASSES, train_imgs, test_imgs, train_labels, test_labels, train_arr, test_arr
    if s == "train":
        imgs = train_imgs
        labels = train_labels
        arr = train_arr
    else:
        imgs = test_imgs
        labels = test_labels
        arr = test_arr
        
    n_classes = NUM_CLASSES
    n_examples, w, h = imgs.shape
    
    classes = list(range(NUM_CLASSES))

    random.shuffle(classes)
    
    true_class = classes[0]
    
    idx1, idx2 = np.random.choice(arr[true_class], size=(2,))
    
    test_image = np.asarray([imgs[idx1,:,:]]*N).reshape(N, w, h, 1)
    
    ssidxs = [idx2]
    for i in range(1, NUM_CLASSES):
        ssidxs.append(np.random.choice(arr[classes[i]]))
    
    support_set = imgs[ssidxs,:,:]
    
    support_set = support_set.reshape(N, w, h, 1)
    
    targets = np.zeros((N,))
    targets[0] = 1
    
    test_image, support_set, targets = shuffle(test_image, support_set, targets)
    
    
    pairs = [test_image, support_set]
    
    return pairs, targets

In [10]:
def test_oneshot(model, N, k, s = "val", verbose = 0):
    n_correct = 0
    if verbose:
        print("Evaluating model on {} random {} way one-shot learning tasks ... \n".format(k,N))
    for i in range(k):
        inputs, targets = make_oneshot_task(N,s)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct+=1
    percent_correct = (100.0 * n_correct / k)
    if verbose:
        print("Got an average of {}% {} way one-shot learning accuracy \n".format(percent_correct,N))
    return percent_correct

In [11]:
test_oneshot(model, 5, 1000, verbose=1)

Evaluating model on 1000 random 5 way one-shot learning tasks ... 

Got an average of 94.4% 5 way one-shot learning accuracy 



94.4